# Installation

In [ ]:
%%shell
git clone https://github.com/oneapi-src/oneCCL.git
cd oneCCL
mkdir build
cd build
cmake ..
make -j install
mkdir -p /opt/intel/oneccl
mv ./_install/env /opt/intel/oneccl

Cloning into 'oneCCL'...
remote: Enumerating objects: 12214, done.
remote: Counting objects: 100% (1032/1032), done.
remote: Compressing objects: 100% (268/268), done.
remote: Total 12214 (delta 879), reused 769 (delta 764), pack-reused 11182 (from 2)
Receiving objects: 100% (12214/12214), 206.95 MiB | 29.11 MiB/s, done.
Resolving deltas: 100% (8629/8629), done.
CMake Deprecation Warning at CMakeLists.txt:18 (cmake_minimum_required):
  Compatibility with CMake < 3.10 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value.  Or, use the <min>...<max> syntax
  to tell CMake that the project requires at least <min> but has been updated
  to work with policies introduced by <max> or earlier.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detec

In [ ]:
#@title Requirements
%%writefile requirements.txt
onnx
onnxruntime
onnx2pytorch
tensorrt
vllm
lmdeploy
openvino
ipex_llm[all]
optimum-intel[extras]
ray
nncf
konlpy
mpi4py
deepspeed

Writing requirements.txt


In [ ]:
#@title Install Packages
%%capture
!pip install intel_extension_for_pytorch==2.1.0 --extra-index-url https://pytorch-extension.intel.com/release-whl/stable/cpu/us/
!pip install https://intel-extension-for-pytorch.s3.amazonaws.com/torch_ccl/cpu/oneccl_bind_pt-2.1.0%2Bcpu-cp39-cp39-linux_x86_64.whl
!VLLM_TARGET_DEVICE=cpu pip install -r -U requirements.txt

In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 85
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 7
microcode	: 0xffffffff
cpu MHz		: 2200.168
cache size	: 39424 KB
physical id	: 0
siblings	: 12
core id		: 0
cpu cores	: 6
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp ibrs_enhanced fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm mpx avx512f avx512dq rdseed adx smap clflushopt clwb avx512cd avx512bw avx512vl xsaveopt xsavec xgetbv1 xsaves arat avx512_vnni md_clear arch_capabilities
bugs		: spectre_v1 spectre_v2 spec_store_bypass swapgs ta

In [ ]:
from google.colab import userdata
import os
os.environ["HF_WRITE_TOKEN"] = userdata.get("HF_WRITE_TOKEN")
os.environ["HUGGINGFACE_API_KEY"] = os.getenv("HF_WRITE_TOKEN")
!huggingface-cli login --token $HF_WRITE_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `WriteToken` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `WriteToken`


# Select Model

In [ ]:
#@title Select Language Model
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import gc

if "origin_model" in globals() or "origin_model" in locals():
    del origin_model

gc.collect()

base_model_id = "beomi/KoAlpaca-KoRWKV-1.5B" # @param ["Gunulhona/tb_pretrained_sts", "Gunulhona/tb_pretrained", "google/flan-t5-xxl", "meta-llama/Meta-Llama-3-8B", "meta-llama/Meta-Llama-3-70B-Instruct", "mistralai/Mistral-7B-Instruct-v0.3", "Qwen/Qwen2-7B-Instruct", "google/gemma-7b", "MLP-KTLim/llama-3-Korean-Bllossom-8B", "EleutherAI/polyglot-ko-12.8b", "vilm/vulture-40b", "arcee-ai/Arcee-Spark", "Qwen/Qwen2-1.5B-Instruct", "OuteAI/Lite-Mistral-150M-v2-Instruct", "google/gemma-2b-it", "beomi/KoAlpaca-KoRWKV-1.5B"] {allow-input: true}

origin_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    trust_remote_code=True,
    # quantization_config=bnb_config
    )

processor = AutoTokenizer.from_pretrained(
    base_model_id,
    add_special_tokens=True,
    trust_remote_code=True)
processor.model_input_names=['input_ids', 'attention_mask']
if processor.pad_token is None:
    processor.pad_token = processor.eos_token

processor.padding_side = "right"
processor.truncation_side = "right"

@torch.inference_mode()
def inference(input_, model):
    template = {
        "role": "user",
        "content": input_
    }
    inputs = processor.apply_to_template(
        template,
        return_tensors='pt'
    )
    # inputs = processor(input_,
    #                    return_tensors="pt",
    #                    padding="max_length",
    #                    truncation=True,
    #                    max_length=128)
    outputs = model.generate(inputs,
                             max_new_tokens=50)
    return processor.batch_decode(outputs, skip_special_tokens=True)[0]

config.json:   0%|          | 0.00/503 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

In [ ]:
#@title Select Vision Model

from transformers import AutoProcessor, Owlv2ForObjectDetection, Owlv2VisionConfig
import torch

base_model_id = "google/owlv2-base-patch16-ensemble" # @param ["google/owlv2-base-patch16-ensemble", ""] {allow-input: true}

processor = AutoProcessor.from_pretrained(base_model_id)
origin_model = Owlv2ForObjectDetection.from_pretrained(base_model_id)

@torch.inference_mode()
def inference(input_, model):
    model.eval()
    inputs = processor(input_, return_tensors="pt")
    outputs = model(**inputs)
    return outputs

# Quantize Model


In [ ]:
from ipex_llm.transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

ipex_model = AutoModelForCausalLM.from_pretrained(base_model_id, trust_remote_code=True)

outputs = inference(input_="test 입력 처리 요구",
                    model=ipex_model)

AttributeError: 'PreTrainedTokenizerFast' object has no attribute 'apply_to_template'

In [ ]:
# prompt: 기존 모델 실행 시간과 onnx qunatization 실행 및 quantized 모델의 실행 시간 비교 하는 코드

import time
import numpy as np
import torch
import onnx
import onnxruntime as ort
import random
import ray
from copy import deepcopy
import string
from pathlib import Path
from onnxruntime.quantization import quantize_dynamic, QuantType
from optimum.onnxruntime import ORTModelForCausalLM, ORTConfig
from transformers.onnx import FeaturesManager
import transformers
import warnings
warnings.filterwarnings("ignore")


@ray.remote
def make_texts():
    return "".join(random.sample('가나다라마바사아자차카타파하그느드르므브스으즈츠크트프흐구누두루무부수우주추쿠투푸후', 30))

randomtexts = ray.get([ make_texts.remote() for i in range(10) ])


# Load your original model
original_model = origin_model
test_in = torch.randint(1000, (1, 128))
with torch.no_grad():
    torch.onnx.export(
        original_model,
        args=tuple(processor(randomtexts[0], return_tensors="pt").values()),
        input_names=["input_ids", "attention_mask"],
        output_names=["logits"],
        dynamic_axes={'input_ids': {0: 'batch_size', 1: 'sequence'},
                      'attention_mask': {0: 'batch_size', 1: 'sequence'},
                      'logits': {0: 'batch_size', 1: 'sequence'}},
        f="original_model.pt")


# Quantize the model using ONNX
quantized_model = quantize_dynamic(
    model_input="original_model.pt",
    model_output="quantized_model.onnx",
    weight_type=onnx.TensorProto.INT8
)

# Save the quantized model
# onnx.save(quantized_model, "quantized_model.onnx")

# Measure execution time for the original model
start_time = time.time()
# Run inference with the original model
result = []
for t in randomtexts:
    outputs = inference(input_=t,
                        model=origin_model)
    result.append(outputs)
end_time = time.time()
original_execution_time = end_time - start_time
print("Original model execution time:", original_execution_time)
print(result[0])

# Measure execution time for the quantized model
start_time = time.time()
# Run inference with the quantized model
q_session = ort.InferenceSession("quantized_model.onnx", providers=["CPUExecutionProvider"])

result = []
for t in randomtexts:
    preprocessed = processor(t, return_tensors="np", padding="max_length", truncation=True, max_length=128)
    onnx_input= {
         "input_ids": preprocessed["input_ids"].astype(np.int64),
         "attention_mask": preprocessed["attention_mask"].astype(np.int64)
     }
    result.append(q_session.run(None, input_feed=onnx_input)[0])
end_time = time.time()
quantized_execution_time = end_time - start_time
print("Quantized model execution time:", quantized_execution_time)
print(result[0].shape)
config = deepcopy(original_model.config)



feature = "causal-lm"
model_kind, model_onnx_config = FeaturesManager.check_supported_model_or_raise(
    original_model, feature=feature)
onnx_config = model_onnx_config(original_model.config)

# export
onnx_inputs, onnx_outputs = transformers.onnx.export(
        preprocessor=processor,
        model=original_model,
        config=onnx_config,
        opset=16,
        output=Path("trfs-model.onnx"))

q_model = ORTModelForCausalLM(q_session, original_model.config,
                              use_io_binding=False, use_cache=False)

# @ray.remote
def ray_onnx(text):
    return inference(input_=text,
                     model=q_model)

start_time = time.time()
result = [ray_onnx(t) for t in randomtexts]
end_time = time.time()
ray_quantized_execution_time = end_time - start_time
print("Ray + Quantized model execution time:", ray_quantized_execution_time)

# Compare execution times
speedup = original_execution_time / quantized_execution_time
print("Speedup:", speedup)
speedup = original_execution_time / ray_quantized_execution_time
print("Ray Speedup:", speedup)


`attention_mask` was passed, but it is unused in this model.
  elem_type: 7
  shape {
    dim {
      dim_value: 3
    }
    dim {
      dim_value: 2
    }
  }
}
.
  elem_type: 7
  shape {
    dim {
      dim_value: 3
    }
    dim {
      dim_value: 2
    }
  }
}
.
  elem_type: 7
  shape {
    dim {
      dim_value: 3
    }
    dim {
      dim_value: 2
    }
  }
}
.
  elem_type: 7
  shape {
    dim {
      dim_value: 3
    }
    dim {
      dim_value: 2
    }
  }
}
.
  elem_type: 7
  shape {
    dim {
      dim_value: 3
    }
    dim {
      dim_value: 2
    }
  }
}
.
  elem_type: 7
  shape {
    dim {
      dim_value: 3
    }
    dim {
      dim_value: 2
    }
  }
}
.
  elem_type: 7
  shape {
    dim {
      dim_value: 3
    }
    dim {
      dim_value: 2
    }
  }
}
.
  elem_type: 7
  shape {
    dim {
      dim_value: 3
    }
    dim {
      dim_value: 2
    }
  }
}
.
  elem_type: 7
  shape {
    dim {
      dim_value: 3
    }
    dim {
      dim_value: 2
    }
  }
}
.
  elem_type:

AttributeError: 'PreTrainedTokenizerFast' object has no attribute 'apply_to_template'

In [ ]:
import deepspeed
from ipex_llm import optimize_model
import os
import time
import warnings
warnings.filterwarnings("ignore")

original_model = origin_model

# Measure execution time for the original model
start_time = time.time()
# Run inference with the original model
result = []
for t in randomtexts:
    outputs = inference(input_=t,
                        model=origin_model)
    result.append(outputs)
end_time = time.time()
original_execution_time = end_time - start_time
print("Original model execution time:", original_execution_time)
print(result[0])


world_size = int(os.getenv("WORLD_SIZE", "2"))
local_rank = int(os.getenv("RANK", "-1")) # RANK is automatically set by CCL distributed backend

ds_model = deepspeed.init_inference(
    original_model, # an AutoModel of Transformers
    mp_size = world_size, # instance (process) count
    dtype=torch.int8,
    replace_method="auto")

ds_model = optimize_model(ds_model.module.to(f'cpu'), low_bit='sym_int4')
ds_model = ds_model.to(f'cpu:{local_rank}') # move partial model to local rank
result = []
with torch.inference_mode():
    start_time = time.time()
    result += [inference(t, ds_model) for t in randomtexts]
    end_time = time.time()

quantized_execution_time = end_time - start_time
print("Deepspeed model execution time:", quantized_execution_time)
print(result[0])

In [ ]:
# prompt: 기존 모델 실행 시간과 tensorrt qunatization 실행 및 quantized 모델의 실행 시간 비교 하는 코드

import time
import torch
import onnx
import ray
import tensorrt as trt
import warnings
warnings.filterwarnings("ignore")

# Load your original model
original_model = origin_model
test_time = 5

# Measure execution time for the original model
start_time = time.time()
# Run inference with the original model
for _ in range(test_time):
    outputs = inference(input_="test 입력 처리 요구",
                        model=origin_model)
end_time = time.time()
original_execution_time = end_time - start_time
print("Original model execution time:", original_execution_time)

# TensorRT quantization and execution
logger = trt.Logger(trt.Logger.INFO)
builder = trt.Builder(logger)
network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))
parser = trt.OnnxParser(network, logger)

with open("quantized_model.onnx", "rb") as model:
    if not parser.parse(model.read()):
        for error in range(parser.num_errors):
            print(parser.get_error(error))

# Build TensorRT engine
engine = builder.build_cuda_engine(network)

# Measure execution time for the TensorRT quantized model
start_time = time.time()
# Run inference with the TensorRT engine
for _ in range(test_time):
    preprocessed = processor("test 입력 처리 요구",
                             return_tensors="np",
                             padding="max_length",
                             truncation=True,
                             max_length=128)
    inputs[0].host = preprocessed["input_ids"].astype(np.float32)  # Assuming input_ids is the input tensor name
    trt_outputs = do_inference(context, bindings=bindings, inputs=inputs, outputs=outputs, stream=stream)
end_time = time.time()
trt_quantized_execution_time = end_time - start_time
print("TensorRT quantized model execution time:", trt_quantized_execution_time)

@ray.remote
def ray_inference(text):
    preprocessed = processor(text,
                            return_tensors="np",
                            padding="max_length",
                            truncation=True,
                            max_length=128)
    inputs[0].host = preprocessed["input_ids"].astype(np.float32)  # Assuming input_ids is the input tensor name
    return do_inference(context, bindings=bindings, inputs=inputs, outputs=outputs, stream=stream)


start_time = time.time()
for _ in range(test_time):
    text_id = ray.put("test 입력 처리 요구")
    ray.get([ray_inference.remote(text_id)])

end_time = time.time()
ray_trt_quantized_execution_time = end_time - start_time
print("Ray + TensorRT quantized model execution time:", ray_trt_quantized_execution_time)


# Compare execution times
speedup_trt = original_execution_time / trt_quantized_execution_time
speedup_ray = original_execution_time / ray_trt_quantized_execution_time
print("Speedup with TensorRT quantization:", speedup_trt)
print("Speedup with Ray + TensorRT :", speedup_ray)


Original model execution time: 179.06584286689758


In [ ]:
# prompt: 기존 모델 실행 시간과 vllm qunatization 실행 및 quantized 모델의 실행 시간 비교 하는 코드
import os
import time
import torch
import onnx
import tensorrt as trt
from vllm import LLM, SamplingParams
import warnings
warnings.filterwarnings("ignore")

os.environ["VLLM_TARGET_DEVICE"] = "cpu"

# Load your original model
original_model = origin_model
test_time = 5

# Measure execution time for the original model
start_time = time.time()
# Run inference with the original model
for _ in range(test_time):
    outputs = inference(input_="test 입력 처리 요구",
                        model=origin_model)
end_time = time.time()
original_execution_time = end_time - start_time
print("Original model execution time:", original_execution_time)

# vllm quantization and execution
# Initialize vLLM with the quantized model
llm = LLM(model=base_model_id,
          tensor_parallel_size=1,
          device='cpu')

# Generate text using vLLM
prompts = ["This is a prompt."]
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)
start_time = time.time()
for _ in range(test_time):
    result = llm.generate(prompts, sampling_params)
end_time = time.time()
vllm_quantized_execution_time = end_time - start_time
print("vLLM quantized model execution time:", vllm_quantized_execution_time)

# Compare execution times
speedup_vllm = original_execution_time / vllm_quantized_execution_time
print("Speedup with vLLM quantization:", speedup_vllm)


Original model execution time: 222.8475422859192
INFO 07-17 15:05:38 llm_engine.py:174] Initializing an LLM engine (v0.5.2) with config: model='Qwen/Qwen2-1.5B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2-1.5B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cpu, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=Qwen/Qwen2-1.5B-Instruct, use_v2_block_manager=False, enable_prefix_caching=False)
WARNING 07-17 15:05:39 cpu_executor.py:134] CUDA graph is not supported on CPU, fallback 

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:

# prompt: 기존 모델 실행 시간과 lmdeploy qunatization 실행 및 quantized 모델의 실행 시간 비교 하는 코드
# 단 lmdeploy는 cli가 아닌 lmdeploy 패키지 import 를 통해 실행하여야 함

import time
import torch
import onnx
import lmdeploy
from onnx2pytorch import ConvertModel
from transformers import AutoConfig, AutoModelForCausalLM
from lmdeploy import turbomind as tm
from lmdeploy import pipeline, TurbomindEngineConfig, ChatTemplateConfig
import warnings
warnings.filterwarnings("ignore")


# Load your original model
original_model = origin_model
test_time = 5

# Measure execution time for the original model
start_time = time.time()
# Run inference with the original model
for _ in range(test_time):
    outputs = inference(input_="test 입력 처리 요구",
                        model=origin_model)
end_time = time.time()
original_execution_time = end_time - start_time
print("Original model execution time:", original_execution_time)

# lmdeploy quantization
onnx_model = onnx.load("quantized_model.onnx" ,load_external_data=True)

torch.save(original_model.model, "hf_model.pth")

# pytorch_model = ConvertModel(onnx_model)
# config = AutoConfig.from_pretrained("bart")  # 적절한 모델 구성으로 변경하세요
# config.architectures = ["BartModel"]  # 모델 아키텍처에 맞게 조정하세요

# hf_model = AutoModelForCausalLM.from_config(config)
# hf_model.load_state_dict(pytorch_model.state_dict())
# hf_model.save_pretrained("onnx_hf_model")

tm_model = tm.TurboMind(model_path=base_model_id)

tm.quantize(
    base_model_id,
    "lmdeploy_quantized.tm",
    w_bits=4, w_group_size=128)
quantized_tm_model = tm.TurboMind(model_path=quantized_model_path)
chat = quantized_tm_model.create_chat_session()

# lmdeploy_pipe = pipeline(
#     base_model_id,
#     backend_config=TurbomindEngineConfig(model_format='hf', tp=4),
#     chat_template_config=ChatTemplateConfig(model_name='llama2'))

# Measure execution time for the lmdeploy quantized model
# Run inference with the quantized model
start_time = time.time()
# Run inference with the original model
for _ in range(test_time):
    response, _ = quantized_chat.chat("This is a prompt.")
end_time = time.time()
lmdeploy_quantized_execution_time = end_time - start_time
print("lmdeploy quantized model execution time:", lmdeploy_quantized_execution_time)

# Compare execution times
speedup_lmdeploy = original_execution_time / lmdeploy_quantized_execution_time
print("Speedup with lmdeploy quantization:", speedup_lmdeploy)


Original model execution time: 29.716737508773804


OSError: Incorrect path_or_model_id: 'Gunulhona/tb_pretrained_sts/triton_models/tokenizer'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
# prompt: 기존 모델 실행 시간과 openvino qunatization 실행 및 quantized 모델의 실행 시간 비교 하는 코드
#@markdown Colab 환경에서는 AMD CPU 사용 중


import numpy as np
import openvino as ov
from optimum.intel import OVModelForCausalLM
from optimum.intel.openvino import OVQuantizationConfig
import warnings
warnings.filterwarnings("ignore")

# Load your original model
original_model = origin_model
test_time = 5

# Measure execution time for the original model
start_time = time.time()
# Run inference with the original model
for _ in range(test_time):
    outputs = inference(input_="test 입력 처리 요구",
                        model=origin_model)
end_time = time.time()
original_execution_time = end_time - start_time
print("Original model execution time:", original_execution_time)

# Load the ONNX model
core = ov.Core()
ov_model = core.read_model("quantized_model.onnx")
ov_model.reshape({model_input.any_name: ov.PartialShape([1, '?']) for model_input in ov_model.inputs})

# Quantize the model
tput = {'PERFORMANCE_HINT': 'THROUGHPUT'}
compiled_model = core.compile_model(ov_model, 'CPU', tput)

# ireqs = ov.AsyncInferQueue(compiled_model)

# ov_model = OVModelForCausalLM.from_pretrained(base_model_id, export=True)
ov_model = OVModelForCausalLM(model=ov_model,
                              config=original_model.config,
                              use_cache=False,
                              use_io_binding=False,)


# Save the quantized model
# ov.serialize(quantized_model, "quantized_model.xml")

# Load the quantized model

# Measure execution time for the OpenVINO quantized model
start_time = time.time()
# Run inference with the OpenVINO quantized model
for _ in range(test_time):
    preprocessed = processor("test 입력 처리 요구",
                             return_tensors="np",
                             padding="max_length",
                             truncation=True,
                             max_length=128)
    onnx_input= {
        "input_ids": preprocessed["input_ids"].astype(np.int64),
        "attention_mask": preprocessed["attention_mask"].astype(np.int64),
     }

    # ireqs.start_async(onnx_input)
    result = inference(input_="test 입력 처리 요구",
                       model=ov_model)
end_time = time.time()
# ireqs.wait_all()
openvino_quantized_execution_time = end_time - start_time
print("OpenVINO quantized model execution time:", openvino_quantized_execution_time)

# Compare execution times
speedup_openvino = original_execution_time / openvino_quantized_execution_time
print("Speedup with OpenVINO quantization:", speedup_openvino)


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Original model execution time: 409.310045003891


Compiling the model to CPU ...
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


OpenVINO quantized model execution time: 45.05198836326599
Speedup with OpenVINO quantization: 9.08528257850723


# AWQ

In [ ]:
!pip install autoawq
!pip install peft
!pip install transformers --upgrade
!pip install av decord
# !git clone https://github.com/kongds/MoRA.git && pip install -e MoRA/peft-mora

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.4 MB/s eta 0:00

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from awq import AutoAWQForCausalLM # installed package is auto awq, but the installed package named awq...
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer
import torch

# model_id = "Gunulhona/Gemma-System-9B-MoRA-SimPO-no-adapter"  # 한국어 모델 ID
model_id = "Qwen/Qwen3-32B"

# model = AutoPeftModelForCausalLM.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, low_cpu_mem_usage=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# model = model.merge_and_unload()

In [ ]:
quant_config = {
    "quant_method": "awq",
    "bits": 4,  # 양자화 비트 수 (예: 4, 8)
    "group_size": 128,  # 그룹 크기 (AWQ 알고리즘의 매개변수)
    "damp_percent": 0.01, # damp percent (AWQ 알고리즘의 매개변수)
    "desc_act": False # desc_act (AWQ 알고리즘의 매개변수)
}

In [ ]:
# 몇 가지 예제 텍스트를 사용하여 calibration 데이터셋을 생성합니다.
calibration_data = [
    "안녕하세요. 오늘 날씨가 어떻습니까?",
    "한국어로 된 질문에 답변해 주세요.",
    "이 모델은 AutoAWQ를 사용하여 양자화되었습니다."
]
inputs = tokenizer(calibration_data, return_tensors="pt", padding=True)

In [ ]:
# Calibration 데이터셋을 사용하여 모델을 양자화합니다.
# Calibration 데이터셋이 없으면 AutoAWQ가 자동으로 생성합니다.

# Qwen2VLAwqQuantizer의 경우, Qwen2VLAwqQuantizer를 사용해야합니다.
# from autoawq.quantizers.qwen2_vl_awq_quantizer import Qwen2VLAwqQuantizer

#quantized_model = AutoAWQForCausalLM.from_pretrained(
#    model_id,
#    quantization_config=quant_config,
#    trust_remote_code=True,
#    quantizer_cls=Qwen2VLAwqQuantizer
#)
# quantized_model = AutoAWQForCausalLM.from_pretrained(
#     model_id,
#     quantization_config=quant_config,
#     trust_remote_code=True,
    # calibration data를 지정할 경우, 아래와 같이 지정합니다.
    # calib_data=inputs
# )

# 모델을 저장합니다.
# quantized_model.save_pretrained("quantized_model")

from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

quant_path = 'quantized-awq'
quant_config = { "zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM" }

# Load model
model = AutoAWQForCausalLM.from_pretrained(model_id, **{"model_type": "hyperclovax_vlm","low_cpu_mem_usage": True})
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

# Quantize
model.quantize(tokenizer, quant_config=quant_config)

# Save quantized model
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)


In [ ]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    inputs = inputs.to(device)
else:
    device = torch.device("cpu")

# 양자화된 모델을 로드합니다.
quantized_model = AutoAWQForCausalLM.from_pretrained(
    "/content/quantized-awq",
    trust_remote_code=True).to(device)

# 추론을 실행합니다.
prompt = "한국어로 된 질문에 답변해 주세요. 어떤 삶을 살아왔나요?"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = quantized_model.generate(
    input_ids=inputs["input_ids"].to(device))
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
# repo_name = "Gunulhona/Gemma-System-9B-MoRA-SimPO-AWQ"
repo_name = "Gunulhona/Qwen3-32B-awq-4bit"
quantized_model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)